In [16]:
from lightning.pytorch import Trainer
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import Dataset

In [ ]:
from pytorch_forecasting.data.data_module import EncoderDecoderTimeSeriesDataModule
from pytorch_forecasting.data.encoders import (
    NaNLabelEncoder,
    TorchNormalizer,
)
from pytorch_forecasting.data.timeseries import TimeSeries
from pytorch_forecasting.metrics import MAE, SMAPE
from pytorch_forecasting.models.temporal_fusion_transformer._tft_v2 import TFT
from pytorch_forecasting.data.examples import load_toydata

In [18]:
num_series = 100
seq_length = 50
data_df = load_toydata(num_series, seq_length)
data_df.head()

,series_id,time_idx,x,y,category,future_known_feature,static_feature,static_feature_cat
0,0,0,-0.093287,0.308908,0,1.000000,0.555166,0
1,0,1,0.308908,0.276330,0,0.995004,0.555166,0
2,0,2,0.276330,0.484302,0,0.980067,0.555166,0
3,0,3,0.484302,0.648938,0,0.955336,0.555166,0
4,0,4,0.648938,0.942956,0,0.921061,0.555166,0


In [19]:
dataset = TimeSeries(
    data=data_df,
    time="time_idx",
    target="y",
    group=["series_id"],
    num=["x", "future_known_feature", "static_feature"],
    cat=["category", "static_feature_cat"],
    known=["future_known_feature"],
    unknown=["x", "category"],
    static=["static_feature", "static_feature_cat"],
)

/content/pytorch-forecasting/pytorch_forecasting/data/timeseries/_timeseries_v2.py:105: UserWarning: TimeSeries is part of an experimental rework of the pytorch-forecasting data layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. For beta testing, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warn(


In [20]:
data_module = EncoderDecoderTimeSeriesDataModule(
    time_series_dataset=dataset,
    max_encoder_length=30,
    max_prediction_length=1,
    batch_size=32,
    categorical_encoders={
        "category": NaNLabelEncoder(add_nan=True),
        "static_feature_cat": NaNLabelEncoder(add_nan=True),
    },
    scalers={
        "x": StandardScaler(),
        "future_known_feature": StandardScaler(),
        "static_feature": StandardScaler(),
    },
    target_normalizer=TorchNormalizer(),
)

/content/pytorch-forecasting/pytorch_forecasting/data/data_module.py:129: UserWarning: TimeSeries is part of an experimental rework of the pytorch-forecasting data layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. For beta testing, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warn(


In [21]:
model = TFT(
    loss=nn.MSELoss(),
    logging_metrics=[MAE(), SMAPE()],
    optimizer="adam",
    optimizer_params={"lr": 1e-3},
    lr_scheduler="reduce_lr_on_plateau",
    lr_scheduler_params={"mode": "min", "factor": 0.1, "patience": 10},
    hidden_size=64,
    num_layers=2,
    attention_head_size=4,
    dropout=0.1,
    metadata=data_module.metadata,
)

print("\nTraining model...")
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=10,
)

trainer.fit(model, data_module)

print("\nEvaluating model...")
test_metrics = trainer.test(model, data_module)

model.eval()
with torch.no_grad():
    test_batch = next(iter(data_module.test_dataloader()))
    x_test, y_test = test_batch
    y_pred = model(x_test)

    print("\nPrediction shape:", y_pred["prediction"].shape)
    print("First prediction values:", y_pred["prediction"][0].cpu().numpy())
    print("First true values:", y_test[0].cpu().numpy())
print("\nTFT model test complete!")

/content/pytorch-forecasting/pytorch_forecasting/models/base/_base_model_v2.py:58: UserWarning: The Model 'TFT' is part of an experimental reworkof the pytorch-forecasting model layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. This class is intended for beta testing and as a basic skeleton, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warn(
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: F


Training model...


INFO: 
  | Name                  | Type               | Params | Mode 
---------------------------------------------------------------------
0 | loss                  | MSELoss            | 0      | train
1 | encoder_var_selection | Sequential         | 709    | train
2 | decoder_var_selection | Sequential         | 193    | train
3 | static_context_linear | Linear             | 192    | train
4 | lstm_encoder          | LSTM               | 51.5 K | train
5 | lstm_decoder          | LSTM               | 50.4 K | train
6 | self_attention        | MultiheadAttention | 16.6 K | train
7 | pre_output            | Linear             | 4.2 K  | train
8 | output_layer          | Linear             | 65     | train
---------------------------------------------------------------------
123 K     Trainable params
0         Non-trainable params
123 K     Total params
0.495     Total estimated model params size (MB)
18        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytor

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.



Evaluating model...


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_MAE          │    0.5018836855888367     │
│        test_SMAPE         │    1.1201428174972534     │
│         test_loss         │   0.012998351827263832    │
└───────────────────────────┴───────────────────────────┘


Prediction shape: torch.Size([32, 1, 1])
First prediction values: [[-0.12206323]]
First true values: [[-0.04691907]]

TFT model test complete!
